In [1]:
server_args = []
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

from manipulation import running_as_notebook

# Imports
import numpy as np
import pydot
from ipywidgets import Dropdown, Layout
from IPython.display import display, HTML, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, AutoDiffXd, autoDiffToGradientMatrix, SnoptSolver,
    initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix,
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint
)

In [2]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)
#add iiwa
iiwa = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
    pydrake.common.FindResourceOrThrow(
        "drake/examples/push_box/iiwa7/iiwa7_with_box_collision.sdf"), "iiwa"
)

plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("iiwa_link_0"))


# Set default positions:
q0 = [0.0, 0.1, 0, -1.2, 0, 1.6, 0]
# q0 = [0.0, 0., 0, 0, 0, 0, 0]
index = 0
for joint_index in plant.GetJointIndices(iiwa):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, pydrake.multibody.tree.RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1


In [3]:
#add gripper
cylinder_gripper = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/cylinder_gripper.sdf"), "cylinder_gripper")

X_7G = RigidTransform(RotationMatrix(), [0, 0, 0.033])
# X_7G = RigidTransform(RollPitchYaw(np.pi / 2.0, 0, np.pi / 2.0), [0, 0, 0.114])
plant.WeldFrames(plant.GetFrameByName("iiwa_link_7", iiwa),
                 plant.GetFrameByName("base_link", cylinder_gripper), X_7G)

<WeldJoint_[float] name='iiwa_link_7_welds_to_base_link' index=8 model_instance=3>

In [4]:
#add table
table = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/extra_heavy_duty_table_surface_only_collision_lower_height.sdf"), "table")

X_WT = RigidTransform(RotationMatrix(), [0.8, 0., 0.])
plant.WeldFrames(plant.world_frame(),
                 plant.GetFrameByName("link", table), X_WT)


#add box
cracker_box = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/003_cracker_box.sdf"), "cracker_box")
#add box_goal
cracker_box_goal = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/003_cracker_box_goal.sdf"), "cracker_box_goal")

X_WBG = RigidTransform(RotationMatrix(), [0.95, 0., 0.3285 + 0.03295])
plant.WeldFrames(plant.world_frame(),
                 plant.GetFrameByName("base_link_cracker", cracker_box_goal), X_WBG)

<WeldJoint_[float] name='WorldBody_welds_to_base_link_cracker' index=10 model_instance=6>

In [5]:
plant.Finalize()
plant.set_name("plant")

In [6]:
iiwa_link_7_body = plant.GetBodyByName("iiwa_link_7", iiwa)
iiwa_link_7_id = plant.GetBodyFrameIdIfExists(iiwa_link_7_body.index())
cylinder_gripper_body = plant.GetBodyByName("base_link", cylinder_gripper)
cylinder_gripper_id = plant.GetBodyFrameIdIfExists(cylinder_gripper_body.index())
cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())
table_link_body = plant.GetBodyByName("link", table)
table_link_id = plant.GetBodyFrameIdIfExists(table_link_body.index())

draw_frames = True
frames_to_draw = [iiwa_link_7_id, cylinder_gripper_id, cracker_box_id, table_link_id] if draw_frames else []
meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, frames_to_draw=frames_to_draw, axis_length=0.15,
                 axis_radius=0.001,)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
X_WB = RigidTransform(RotationMatrix(), [0.55, 0., 0.3285 + 0.03295])
plant.SetFreeBodyPose(plant_context, 
                              plant.GetBodyByName("base_link_cracker", cracker_box),
                              X_WB)

meshcat.load()
diagram.Publish(context)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6013...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7013/static/
Connected to meshcat-server.


In [7]:
# Make the plant for the MathematicalProgram to use.
prog_iiwa_plant = pydrake.multibody.plant.MultibodyPlant(time_step=1e-4)
prog_iiwa =  pydrake.multibody.parsing.Parser(prog_iiwa_plant).AddModelFromFile(
    pydrake.common.FindResourceOrThrow(
        "drake/examples/push_box/iiwa7/iiwa7_with_box_collision.sdf"), "prog_iiwa"
)
prog_iiwa_plant.WeldFrames(prog_iiwa_plant.world_frame(), prog_iiwa_plant.GetFrameByName("iiwa_link_0"))

#add gripper
prog_cylinder_gripper = pydrake.multibody.parsing.Parser(prog_iiwa_plant).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/cylinder_gripper.sdf"), "prog_cylinder_gripper")

X_7G = RigidTransform(RotationMatrix(), [0, 0, 0.033])
prog_iiwa_plant.WeldFrames(prog_iiwa_plant.GetFrameByName("iiwa_link_7", prog_iiwa),
                 prog_iiwa_plant.GetFrameByName("base_link", prog_cylinder_gripper), X_7G)

prog_iiwa_plant.Finalize()
prog_iiwa_plant.set_name("prog_iiwa_plant")




prog_box_plant = pydrake.multibody.plant.MultibodyPlant(time_step=1e-4)
#add table
prog_table = pydrake.multibody.parsing.Parser(prog_box_plant).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/extra_heavy_duty_table_surface_only_collision_lower_height.sdf"), "prog_table")

X_WT = RigidTransform(RotationMatrix(), [0.8, 0., 0.])
prog_box_plant.WeldFrames(prog_box_plant.world_frame(),
                 prog_box_plant.GetFrameByName("link", prog_table), X_WT)


#add box
prog_cracker_box = pydrake.multibody.parsing.Parser(prog_box_plant).AddModelFromFile(
            pydrake.common.FindResourceOrThrow(
                "drake/examples/push_box/003_cracker_box.sdf"), "prog_cracker_box")

prog_box_plant.Finalize()
prog_box_plant.set_name("prog_box_plant")

In [10]:
mu = 1 # rubber on rubber
total_mass = sum(plant.get_body(index).get_mass(context) for index in plant.GetBodyIndices(cracker_box))
gravity = plant.gravity_field().gravity_vector()
finger_point_frame = plant.GetFrameByName("finger_point", cylinder_gripper)

#set contact sequence
N = 100
in_contact = np.zeros((N))
in_contact[40:] = 1

T = 5
prog = MathematicalProgram()
# Time steps    
h = prog.NewContinuousVariables(N-1, "h")
prog.AddBoundingBoxConstraint(0.5*T/N, 2.0*T/N, h) 
prog.AddLinearConstraint(sum(h) >= .9*T)
prog.AddLinearConstraint(sum(h) <= 1.1*T)

# Create one context per timestep (to maximize cache hits)
plant_context_list = [plant.CreateDefaultContext() for i in range(N)]
# We could get rid of this by implementing a few more Jacobians in MultibodyPlant:
ad_plant = plant.ToAutoDiffXd()

nq_r = plant.num_positions(iiwa)
q_r = prog.NewContinuousVariables(nq_r, N, "q_r")
v_r = prog.NewContinuousVariables(nq_r, N, "v_r")

q0 = plant.GetPositions(plant_context, iiwa)
for n in range(N):
    # Joint limits
    prog.AddBoundingBoxConstraint(prog_iiwa_plant.GetPositionLowerLimits(), prog_iiwa_plant.GetPositionUpperLimits(), q_r[:,n])
    # Joint velocity limits
    prog.AddBoundingBoxConstraint(prog_iiwa_plant.GetVelocityLowerLimits(), prog_iiwa_plant.GetVelocityUpperLimits(), v_r[:,n])

    # Initial guess for all joint angles is the home position
    prog.SetInitialGuess(q_r[:,n], q0)  

    # Running costs:
    prog.AddQuadraticErrorCost(np.diag([1]*nq_r), q0, q_r[:,n])
    prog.AddQuadraticErrorCost(np.diag([1]*nq_r), [0]*nq_r, v_r[:,n])

#position and velocity relation constrain
for n in range(N-1):
    prog.AddConstraint(eq(q_r[:, n+1], q_r[:,n] + h[n]*v_r[:,n]))

    
u = prog.NewContinuousVariables(nq_r, N, "u")
#TODO: find iiwa real torque limits 
u_limits = np.array([100, 100, 100, 30, 30, 30, 30])
for n in range(N):
    # Joint torque limits
    prog.AddBoundingBoxConstraint(-u_limits, u_limits, u[:,n])
     # Running costs:
    prog.AddQuadraticErrorCost(np.diag([1]*nq_r), [0]*nq_r, u[:,n])

#relate to box frame
f = prog.NewContinuousVariables(3, N, "f")
#why only to N-1
for n in range(N-1):
    # Linear friction cone
    prog.AddLinearConstraint(f[1,n] <= mu*f[0,n])
    prog.AddLinearConstraint(-f[1,n] <= mu*f[0,n])
    prog.AddLinearConstraint(f[2,n] <= mu*f[0,n])
    prog.AddLinearConstraint(-f[2,n] <= mu*f[0,n])
    # normal force >=0, normal_force == 0 if not in_contact
    prog.AddBoundingBoxConstraint(0, in_contact[n]*4*9.81*total_mass, f[0,n])
    


In [ ]:
# Kinematic constraints
def fixed_position_constraint(vars, context_index, frame):
    q, qn = np.split(vars, [nq])
    if not np.array_equal(q, plant.GetPositions(context[context_index])):
        plant.SetPositions(context[context_index], q)
    if not np.array_equal(qn, plant.GetPositions(context[context_index+1])):
        plant.SetPositions(context[context_index+1], qn)
    p_WF = plant.CalcPointsPositions(context[context_index], frame, [0,0,0], plant.world_frame())
    p_WF_n = plant.CalcPointsPositions(context[context_index+1], frame, [0,0,0], plant.world_frame())
    if isinstance(vars[0], AutoDiffXd):
        J_WF = plant.CalcJacobianTranslationalVelocity(context[context_index], JacobianWrtVariable.kQDot,
                                                frame, [0, 0, 0], plant.world_frame(), plant.world_frame())
        J_WF_n = plant.CalcJacobianTranslationalVelocity(context[context_index+1], JacobianWrtVariable.kQDot,
                                                frame, [0, 0, 0], plant.world_frame(), plant.world_frame())
        return initializeAutoDiffGivenGradientMatrix(
            p_WF_n - p_WF, J_WF_n @ autoDiffToGradientMatrix(qn) - J_WF @ autoDiffToGradientMatrix(q))
    else:
        return p_WF_n - p_WF
for i in range(4):
    for n in range(N):
        if in_stance[i, n]:
            # foot should be on the ground (world position z=0)
            prog.AddConstraint(PositionConstraint(
                plant, plant.world_frame(), [-np.inf,-np.inf,0], [np.inf,np.inf,0], 
                foot_frame[i], [0,0,0], context[n]), q[:,n])
            if n > 0 and in_stance[i, n-1]:
                # feet should not move during stance.
                prog.AddConstraint(partial(fixed_position_constraint, context_index=n-1, frame=foot_frame[i]),
                                   lb=np.zeros(3), ub=np.zeros(3), vars=np.concatenate((q[:,n-1], q[:,n])))
        else:
            min_clearance = 0.01
            prog.AddConstraint(PositionConstraint(plant, plant.world_frame(), [-np.inf,-np.inf,min_clearance], [np.inf,np.inf,np.inf],foot_frame[i],[0,0,0],context[n]), q[:,n])
